# Wildfire Visualization and OpenEO


This notebook demonstrates how to visualize wildfire occurrences using Sentinel-2 imagery. The visualization applies cartographic-style rendering that the author can switch between different purposes.

The available styles include:
1. A continuous heatmap-like visualization where fire intensity (from weak to strong) is encoded using SWIR-based thresholds and blended into a base map that preserves soil, built-up areas, vegetation, and water bodies.
2. Binary fire detection styles using read or yellow coloring to indicate the presence or absence of active fires.
3. Context-preserving binary styles that highlight fire pixels while maintaining the underlying land-cover information for situational awareness.

The visualization also includes optional cloud avoidance logic to reduce false detections, and supports burn-scar highlighting using the Normalized Burn Ration (NBR) to visualize post-fire impacts. Other enhancements, such as contrast and saturation, are applied to beautify the visualization. 

ToDo:
- Create the function for different viz style
- Integrate this different viz style to `wildfire_viz_helper.py`
- Test the process
- Documentation: explaining what the notebook does, the modularity and the reasoning, provide other example usage, add credits

In [1]:
import openeo
import matplotlib.pyplot as plt
from PIL import Image
from openeo.processes import array_create, if_, and_, array_element
from wildfire_viz_helper import stretch, satEnh, layer_blend, isCloud, not_, or_
from wildfire_viz_color_arrays import naturalColorsCC_arr, naturalColors_arr, urban_arr

# Connect to OpenEO Backend
Connect to the OpenEO backend and authenticate using OpenID Connect.

In [2]:
# connection = openeo.connect(
#     url="https://openeo.dataspace.copernicus.eu/"
# ).authenticate_oidc()

In [3]:
connection = openeo.connect(
    url="https://api.explorer.eopf.copernicus.eu/openeo"
    # url="http://127.0.0.1:8081/"
).authenticate_oidc_authorization_code()

# Define Area of Interest
Define the spatial extent for our analysis. This example uses coordinates for Battleship Mountain, B.C., Canada

In [4]:
spatial_extent = {
    "west": 14.882795218158861,
    "east": 14.913869863464043,
    "south": 44.84231291017926,
    "north": 44.859776923695904
}

# Load Sentinel-2 Data
Load Sentinel-2 L1C data. We need multiple bands to visualize the underlying land-cover information, water, and wildfire.
- **B01**: Coastal aerosol, 442.7 nm
- **B02**: Blue, 492.4 nm
- **B03**: Green, 559.8 nm
- **B04**: Red, 664.6 nm
- **B08**: NIR, 832.8 nm
- **B8A**: Narrow NIR, 864.7 nm
- **B11**: SWIR, 1613.7 nm
- **B12**: SWIR, 2202.4 nm
- **CLP**: Cloud probability, ranging from 0-255 (divide by 255 to get to the [0-1] range)
- **dataMask**: The mask of data/no data pixels. No data pixels are 1) all pixels which lay outside of the requested polygon, 2) all pixels for which no source data was found, and 3) all pixels for which source data was found and is explicitly "no data"

In [5]:
# s2cube = connection.load_collection(
#     "SENTINEL2_L1C",
#     spatial_extent=spatial_extent,
#     temporal_extent=["2022-09-10", "2022-09-10"],
#     bands=[
#         # "B01", 
#         "B02",
#         "B03",
#         "B04",
#         "B08",
#         # "B8A",
#         "B11",
#         "B12",
#         # "dataMask"
#     ],
# )

# # Resample to 60m (for test)
# # s2cube = s2cube.resample_spatial(resolution=100)

# # # Filter for a specific date
# # s2cube = s2cube.reduce_dimension(dimension="t", reducer="first")å

# # Scale bands from digital numbers (0-10000) to reflectance (0-1)
# s2cube = s2cube.apply(lambda x: x / 10000.0)

In [6]:
s2cube = connection.load_collection(
    "sentinel-2-l2a",
    spatial_extent=spatial_extent,
    temporal_extent=["2025-07-04", "2025-07-10"],
    bands=[f"reflectance|b02", "reflectance|b03", "reflectance|b04", "reflectance|b08", "reflectance|b11", "reflectance|b12"]
)

s2cube = s2cube.reduce_dimension(dimension="time", reducer="first")

In [7]:
# s2cube = connection.datacube_from_process(
#     "load_zarr",
#     url="s3://esa-zarr-sentinel-explorer-fra/tests-output/sentinel-2-l2a/S2B_MSIL2A_20250524T100029_N0511_R122_T33TVK_20250524T125315.zarr/measurements/reflectance",
#     spatial_extent={
#         "east" : spatial_extent["east"],
#         "north" : spatial_extent["north"],
#         "west" : spatial_extent["west"],
#         "south" : spatial_extent["south"]
#     },
#     options={
#         "variables": [
#             "/measurements/reflectance/b02",
#             "/measurements/reflectance/b03",
#             "/measurements/reflectance/b04",
#             "/measurements/reflectance/b08",
#             "/measurements/reflectance/b11",
#             "/measurements/reflectance/b12"
#         ],
#         "width" : 1024,
#         "height" : 1024
#     }
# )

# Wildfire Visualization
- Pick a selection of bands
- Set up the constants
- Calculate indexes for vegetation, water, hotspots and burnt areas
- Define the visualization based on different styles

In [8]:
# If user wants to visualize the hotspots, they can set the hotspot variable to 1
hotspot = 1

# If user wants to visualize the hotspots in a continuous heatmap-style, they can set the style variable to 1
style = 1

# If user wants to visualize the burn scars, they can set the showBurnScars variable to 1, and hotspot to 0
showBurnScars = 0

# Threshold for hotspot intensity
hsThreshold = [2.0, 1.5, 1.25, 1.0]
hsSensitivity = 1.0
boost = 1

# Threshold for cloud probability
cloudAvoidance = 1
cloudAvoidanceThreshold = 245
avoidanceHelper = 0.8

# Image brightness, saturation, and contrast
offset = -0.000
saturation = 1.10
brightness = 1.00
sMin = 0.01
sMax = 0.99

# Threshold for burn scars
burnScarThreshold = -0.25
burnScarStrength = 0.3

# Threshold for water
waterHighlight = 0
waterBoost = 2.0
waterHelper = 0.2

# Index thresholds
NDVI_threshold = -0.15
NDWI_threshold = 0.15


In [9]:
def wildfire_viz_style(base_arr, b02, b03, b04, b11, b12, b08=None):
    if hotspot:
        base_r = array_element(base_arr, 0)
        base_g = array_element(base_arr, 1)
        base_b = array_element(base_arr, 2)

        clp = isCloud(b03, b04)

        cloud_mask = and_(clp < cloudAvoidanceThreshold, b02 < avoidanceHelper)

        swir_sum = b12 + b11

        if style == 1:
            mask_0 = and_(
                cloud_mask, 
                (swir_sum > (hsThreshold[0] / hsSensitivity))
            )

            mask_1 = and_(
                cloud_mask,
                and_(
                    swir_sum > (hsThreshold[1] / hsSensitivity),
                    not_(mask_0)
                )
            )

            mask_2 = and_(
                cloud_mask,
                and_(
                    swir_sum > (hsThreshold[2] / hsSensitivity),
                    not_(or_(mask_0, mask_1))
                )
            )

            mask_3 = and_(
                cloud_mask,
                and_(
                    swir_sum > (hsThreshold[3] / hsSensitivity),
                    not_(or_(mask_0, or_(mask_1, mask_2)))
                )
            )

            # Any hotspot present
            mask_any = or_(mask_0, or_(mask_1, or_(mask_2, mask_3)))

            # Any hotspot must be colored red
            r = base_r + (boost * 0.5 * b12 * mask_any)

            # Weights green channel by hotspot classes
            g = base_g + (
                boost * b11 * (
                    0.5 * mask_0 + 
                    0.2 * mask_1 + 
                    0.1 * mask_2 + 
                    0.0 * mask_3
                )
            )

            # Blue channel remains unchanged
            b = base_b

            return array_create([r, g, b])

        # Different styles for binary colors
        fire_mask = and_(cloud_mask, (swir_sum > (hsThreshold[3] / hsSensitivity)))
        if style == 2:
            return array_create([
                if_(fire_mask, 1, base_r),
                if_(fire_mask, 0, base_g),
                if_(fire_mask, 0, base_b)
            ])
        
        if style == 3:
            return array_create([
                if_(fire_mask, 1, base_r),
                if_(fire_mask, 1, base_g),
                if_(fire_mask, 0, base_b)
            ])

        if style == 4:
            return array_create([
                if_(fire_mask, base_r + 0.2, base_r),
                if_(fire_mask, base_g - 0.2, base_g),
                if_(fire_mask, base_b - 0.2, base_b)
            ])
            
    if showBurnScars:
        if b08 is None:
            raise ValueError("B08 is required when showBurnScars is True")

        NBR = (b08 - b12) / (b08 + b12)
        NBR_mask = NBR < burnScarThreshold
        
        r = if_(
            NBR_mask, 
            base_r + burnScarStrength,
            base_r
        )

        g = if_(
            NBR_mask,
            base_g + burnScarStrength,
            base_g
        )

        b = base_b

        return array_create([r, g, b])
    
    return base_arr

In [ ]:
def wildfire_visualization(data):
    # Extract bands, remove band 1 and band 8a
    B02, B03, B04, B08, B11, B12 = (
        data[0],
        data[1],
        data[2],
        data[3],
        data[4],
        data[5],
    )

    # Calculate indexes for vegetation, water, and burnt areas
    NDWI = (B03 - B08) / (B03 + B08)
    NDVI = (B08 - B04) / (B08 + B04)

    # True colors with color correction
    naturalColorsCC = naturalColorsCC_arr(brightness, B04, B03, B02, offset)

    # True colors
    naturalColors = naturalColors_arr(brightness, B04, B03, B02, offset)

    # SWIR composite for urban areas
    urban = urban_arr(brightness, B12, B11, B04, offset)

    # Create wildfire visualization base map as an RGB combination from URBAN, naturalColors, naturalColorsCC
    # User can choose to use other color arrays already defined above to create the visualization base map
    baseViz = layer_blend(urban, naturalColors, naturalColorsCC, 10, 40, 50)

    # Coloring the water
    baseViz_red = array_element(baseViz, 0)

    water_detection = and_(and_(NDVI < NDVI_threshold, NDWI > NDWI_threshold), B04 < waterHelper)

    baseViz_green = if_(
        waterHighlight,
        if_(
            water_detection,
            array_element(baseViz, 1) * 1.2 * waterBoost + 0.1,
            array_element(baseViz, 1)
        ),
        array_element(baseViz, 1)
    )

    baseViz_blue = if_(
        waterHighlight,
        if_(
            water_detection,
            array_element(baseViz, 2) * 1.5 * waterBoost + 0.2,
            array_element(baseViz, 2)
        ),
        array_element(baseViz, 2)
    )

    baseViz = array_create([baseViz_red, baseViz_green, baseViz_blue])

    # Enhance saturation and contrast
    baseViz = satEnh(baseViz, saturation)
    baseViz = stretch(baseViz, sMin, sMax)

    # Pick viz style
    baseViz = wildfire_viz_style(baseViz, B02, B03, B04, B11, B12)

    return baseViz

In [11]:
wildfire_viz_image = s2cube.apply_dimension(dimension="bands", process=wildfire_visualization)

wildfire_viz_image = wildfire_viz_image.linear_scale_range(input_min=0, input_max=1, output_min=0, output_max=255)

wildfire_viz_image = wildfire_viz_image.save_result("PNG")

In [12]:
wildfire_viz_image.download("wildfire_hotspot_style2.png")

Preflight process graph validation failed: [400] InvalidRequest: 1 validation error:
  {'type': 'missing', 'loc': ('body', 'id'), 'msg': 'Field required', 'input': {'process_graph': {'loadcollection1': {'process_id': 'load_collection', 'arguments': {'bands': ['reflectance|b02', 'reflectance|b03', 'reflectance|b04', 'reflectance|b08', 'reflectance|b11', 'reflectance|b12'], 'id': 'sentinel-2-l2a', 'spatial_extent': {'west': 14.882795218158861, 'east': 14.913869863464043, 'south': 44.84231291017926, 'north': 44.859776923695904}, 'temporal_extent': ['2025-07-04', '2025-07-10']}}, 'reducedimension1': {'process_id': 'reduce_dimension', 'arguments': {'data': {'from_node': 'loadcollection1'}, 'dimension': 'time', 'reducer': {'process_graph': {'first1': {'process_id': 'first', 'arguments': {'data': {'from_parameter': 'data'}}, 'result': True}}}}}, 'applydimension1': {'process_id': 'apply_dimension', 'arguments': {'data': {'from_node': 'reducedimension1'}, 'dimension': 'bands', 'process': {'proc

OpenEoApiError: [500] ServerError: "Task execution failed for key 'S2C_MSIL2A_20250708T100051_N0511_R122_T33TVK_20250708T155705'"